In [1]:
! pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
qa_dataset = load_dataset("mpingale/mental-health-chat-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/840 [00:00<?, ?B/s]

data/train-00000-of-00001-991edb316b3098(…):   0%|          | 0.00/3.64M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

In [ ]:
# save_path = "/content/drive/MyDrive/loaded_dataset/dude-sample-qas"
# qa_dataset.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7 [00:00<?, ? examples/s]

In [3]:
# from datasets import load_from_disk
# qa_dataset = load_from_disk("/content/drive/MyDrive/loaded_dataset/dude-sample-qas")

In [3]:
train_qas = qa_dataset["train"]

print(train_qas[0])

{'questionID': 0, 'questionTitle': 'Do I have too many issues for counseling?', 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling', 'topic': 'depression', 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor', 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari', 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be h

In [4]:
qa_dataset["train"][0]["questionText"]

'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?'

In [5]:
from collections import defaultdict
import json

In [6]:
def prepare_dataset(dataset):
    # لكل سؤال هنجمع: العنوان، النص، وكل الإجابات
    merged = defaultdict(lambda: {
        "questionTitle": None,
        "questionText": None,
        "all_answers": []
    })

    for item in dataset:
        qid = item.get("questionID")
        answer = item.get("answerText")

        # تخطي أي عنصر ناقص QID أو Answer
        if qid is None or not answer:
            continue

        # حفظ العنوان والنص لأول مرة فقط
        if merged[qid]["questionTitle"] is None:
            merged[qid]["questionTitle"] = item.get("questionTitle")
            merged[qid]["questionText"] = item.get("questionText")

        # تنظيف الإجابة
        clean_answer = answer.replace("\xa0", " ").strip()

        # إضافة الإجابة لو مش مكررة
        if clean_answer not in merged[qid]["all_answers"]:
            merged[qid]["all_answers"].append(clean_answer)

    # تحويل الدكشنري لقائمة نهائية
    prepared_dataset = []
    for qid, v in merged.items():
        combined_answer = "\n\n---\n\n".join(v["all_answers"])  # فاصل بين الإجابات
        prepared_dataset.append({
            "questionID": qid,
            "questionTitle": v["questionTitle"],
            "questionText": v["questionText"],
            "combined_answer": combined_answer
        })

    return prepared_dataset


In [7]:
# تشغيل الدالة
prepared_dataset = prepare_dataset(train_qas)

In [8]:
# طباعة أول 5 عناصر
for item in prepared_dataset[:5]:
    print(json.dumps(item, indent=2, ensure_ascii=False))
    print("=" * 120)

{
  "questionID": 0,
  "questionTitle": "Do I have too many issues for counseling?",
  "questionText": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "combined_answer": "It is very common for people to have multiple issues that they want to (and need to) address in counseling.  I have had clients ask that same question and through more exploration, there is often an underlying fear that they  \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,  people have numerous significant stressors in their lives.  L

In [9]:
final_dataset = []

for item in prepared_dataset:
    question = item["questionText"] or item["questionTitle"]
    answer = item["combined_answer"]

    final_dataset.append({
        "instruction": question.strip(),
        "response": answer.strip()
    })

# طباعة أول 5 عينات
import json
for sample in final_dataset[:5]:
    print(json.dumps(sample, indent=2, ensure_ascii=False))
    print("=" * 80)

{
  "instruction": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "response": "It is very common for people to have multiple issues that they want to (and need to) address in counseling.  I have had clients ask that same question and through more exploration, there is often an underlying fear that they  \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,  people have numerous significant stressors in their lives.  Let's face it, life can be complicated! Therapists are completely ready and equipped to hand

In [10]:
len(final_dataset)

937

# cleaning the dataset

In [11]:
import re
import html

In [12]:
def clean_text(text):
    if not text:
        return ""

    text = html.unescape(text)

    # إزالة الرموز والإيموجي
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)

    # إزالة HTML أو markdown links
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)

    # إزالة \xa0
    text = text.replace("\xa0", " ")

    # إزالة فواصل أو فواصل طويلة مثل --- أو ===
    text = re.sub(r'-{3,}|={3,}', ' ', text)

    # إضافة مسافة بعد النقطة لو ناقصة
    text = re.sub(r'\.(\w)', r'. \1', text)

    # إزالة المسافات الزائدة قبل علامات الترقيم
    text = re.sub(r'\s+([,.:;?!])', r'\1', text)

    # توحيد المسافات
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [13]:
cleaned_dataset = []

for item in final_dataset:
    q = item["instruction"]
    a = item["response"]

    cleaned_dataset.append({
        "instruction": clean_text(q),
        "response": clean_text(a)
    })


In [14]:
import json

for sample in cleaned_dataset[:5]:
    print(json.dumps(sample, indent=2, ensure_ascii=False))
    print("=" * 80)


{
  "instruction": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "response": "It is very common for people to have multiple issues that they want to (and need to) address in counseling. I have had clients ask that same question and through more exploration, there is often an underlying fear that they \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not, people have numerous significant stressors in their lives. Let's face it, life can be complicated! Therapists are completely ready and equipped to handle all of t

In [15]:
max_words = 0
max_item = None

for item in cleaned_dataset:
    # نعد الكلمات في الـ response
    word_count = len(item["response"].split())

    if word_count > max_words:
        max_words = word_count
        max_item = item

print("أكبر عدد كلمات:", max_words)
print("="*80)
print("العنصر صاحب أكبر إجابة:")
import json
print(json.dumps(max_item, indent=2, ensure_ascii=False))


أكبر عدد كلمات: 15371
العنصر صاحب أكبر إجابة:
{
  "instruction": "How do you know you have the right therapist for you? How would I know how to \"train\" my therapist to be able to give me what I need from treatment?",
  "response": "Finding the right therapist for you is very important and can sometimes be tricky. It can sometimes take a number of sessions to get a good sense of whether you and your therapist are the \"right fit.\" The first couple of sessions are generally spent on gathering information, formulating a plan of treatment, and building the client/therapist relationship. The client/therapist relationship will be very different from other relationships you have experienced. You will know you have found the right therapist when you notice there is a good rapport between the two of you. You will get a sense that the therapist \"gets you\" and understands the issues being presented. If you feel that you can trust your therapist and feel comfortable opening up and providing f

In [16]:
# حساب طول كل إجابة
lengths = [(len(item["response"].split()), item) for item in cleaned_dataset]

# ترتيبهم تنازلي
lengths_sorted = sorted(lengths, key=lambda x: x[0], reverse=True)

# عرض أول 5
for count, item in lengths_sorted[:5]:
    print(f"عدد كلمات: {count}")
    print(json.dumps(item, indent=2, ensure_ascii=False))
    print("="*80)


عدد كلمات: 15371
{
  "instruction": "How do you know you have the right therapist for you? How would I know how to \"train\" my therapist to be able to give me what I need from treatment?",
  "response": "Finding the right therapist for you is very important and can sometimes be tricky. It can sometimes take a number of sessions to get a good sense of whether you and your therapist are the \"right fit.\" The first couple of sessions are generally spent on gathering information, formulating a plan of treatment, and building the client/therapist relationship. The client/therapist relationship will be very different from other relationships you have experienced. You will know you have found the right therapist when you notice there is a good rapport between the two of you. You will get a sense that the therapist \"gets you\" and understands the issues being presented. If you feel that you can trust your therapist and feel comfortable opening up and providing feedback during your sessions 

In [17]:
import json

with open("cleaned_dataset.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_dataset, f, ensure_ascii=False, indent=2)

# LangChain JSONLoader

In [18]:
!pip install -q -U langchain langchain-community sentence-transformers faiss-cpu jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Load & Inspect

In [19]:
# طريقة (أ) باستخدام JSONLoader من langchain_community
try:
    from langchain_community.document_loaders import JSONLoader
    using_jsonloader = True
except Exception as e:
    print("langchain_community JSONLoader غير متاح:", e)
    using_jsonloader = False

import json
from pathlib import Path

JSON_PATH = "/content/cleaned_dataset.json"  # غيّر الاسم لو ملفك باسم تاني

def inspect_using_jsonloader(path):
    loader = JSONLoader(file_path=path, jq_schema=".[]", text_content=False)
    docs = loader.load()
    print("=== JSONLoader: عدد الـ docs ===", len(docs))
    # اطبع أول 3 docs محتوى و metadata
    for i, d in enumerate(docs[:3]):
        print(f"\n--- doc {i} ---")
        # قد تكون الصفحة في d.page_content أو في metadata
        pc = getattr(d, "page_content", None)
        print("page_content:", pc)
        print("metadata keys:", list(d.metadata.keys()))
    return docs

def inspect_manually(path):
    raw = Path(path).read_text(encoding="utf-8")
    try:
        data = json.loads(raw)
    except Exception as e:
        print("خطأ في قراءه JSON:", e)
        raise

    # نوع البنية
    print("نوع الـ JSON الجذري:", type(data))
    if isinstance(data, list):
        print("عدد السجلات (list length):", len(data))
    else:
        # object
        print("الكائن الجذري keys:", list(data.keys())[:30])

    # اطبع أول 3 عناصر مع عرض الحقول الرئيسية
    sample_count = min(3, len(data) if isinstance(data, list) else 1)
    for i in range(sample_count):
        item = data[i] if isinstance(data, list) else data
        print(f"\n--- sample {i} ---")
        # اطبع المفاتيح المتاحة
        print("keys:", list(item.keys()))
        # اطبع الحقول المهمة (trim لو طويل)
        for k in ["instruction", "response", "input", "output", "text"]:
            if k in item:
                v = item[k]
                print(f"{k} (len={len(str(v))}):", str(v)[:400])
        # لو فيه مفاتيح غير متوقعة اطبع أول جزئية منهم
        other_keys = [k for k in item.keys() if k not in ("instruction","response","input","output","text")]
        if other_keys:
            print("other keys (examples):", other_keys[:10])
    return data

# تنفيذ الفحص
if using_jsonloader:
    try:
        docs = inspect_using_jsonloader(JSON_PATH)
    except Exception as e:
        print("فشل JSONLoader. هنحاول التحميل اليدوي. error:", e)
        data = inspect_manually(JSON_PATH)
else:
    data = inspect_manually(JSON_PATH)


=== JSONLoader: عدد الـ docs === 937

--- doc 0 ---
page_content: {"instruction": "I have so many issues to address. I have a history of sexual abuse, I\u2019m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I\u2019m beginning to have anxiety. I have low self esteem but I\u2019ve been happily married for almost 35 years. I\u2019ve never had counseling about any of this. Do I have too many issues to address in counseling?", "response": "It is very common for people to have multiple issues that they want to (and need to) address in counseling. I have had clients ask that same question and through more exploration, there is often an underlying fear that they \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not, people have numerous significant stressors in their lives. Let's face it, life ca

# cleaning & anonymization

In [20]:
import json
import re
from collections import Counter

# افتراض: `docs` هو اللي رجعه JSONLoader (قائمة من Document objects)
# لو عندك بدالها raw_data من inspect_manually، أبقي قولي وأأكد الكود

# ---------- تعاريف للتعمية (أمثلة بسيطة) ----------
EMAIL_RE = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
PHONE_RE = re.compile(r'(\+?\d[\d\-\s]{6,}\d)')  # مبسطة
NAME_RE = re.compile(r'\b([A-Z][a-z]{2,}(?:\s[A-Z][a-z]{2,}){0,2})\b')  # تخمين أسماء إنجليزية - استعمل بحذر

def redact_pii(text):
    # استبدل إيميل و أرقام
    text = EMAIL_RE.sub("[REDACTED_EMAIL]", text)
    text = PHONE_RE.sub("[REDACTED_PHONE]", text)
    # مثال: استبدال بعض الأسماء المحتملة — اختياري، ممكن يوقع في false positives
    # text = NAME_RE.sub("[REDACTED_NAME]", text)
    return text

# ---------- تحويل الـ docs ----------
processed = []
stats = Counter()

for i, d in enumerate(docs):
    pc = getattr(d, "page_content", None)
    meta = dict(getattr(d, "metadata", {}) or {})
    parsed = None

    # page_content هنا غالباً string JSON -> نحاول نفكه
    if pc:
        try:
            parsed = json.loads(pc)
        except Exception:
            # لو مش JSON صالح، خليه نص عادي
            parsed = {"instruction": str(pc)}

    # بعض الـ loaders ممكن يحطوا الحقول في metadata بدل page_content
    if not parsed:
        parsed = meta.copy()

    instruction = parsed.get("instruction") or parsed.get("input") or parsed.get("text") or ""
    response = parsed.get("response") or parsed.get("output") or ""

    instruction = str(instruction).strip()
    response = str(response).strip()

    # تعيين flags لإحصاءات
    if not instruction:
        stats["missing_instruction"] += 1
    if not response:
        stats["missing_response"] += 1

    # تعمية بسيطة
    instruction_redacted = redact_pii(instruction)
    response_redacted = redact_pii(response)

    # نص واحد مناسب للـ RAG
    doc_text = f"Instruction: {instruction_redacted}\nResponse: {response_redacted}"

    # metadata نهائي
    final_meta = {
        "source": meta.get("source"),
        "seq_num": meta.get("seq_num"),
        "orig_length": len(instruction) + len(response),
        "has_response": bool(response),
    }
    # دمج أى meta إضافي لو حابب
    for k in ("source","seq_num"):
        if k in meta:
            final_meta[k] = meta[k]

    processed.append({
        "text": doc_text,
        "metadata": final_meta
    })

    # بعض احصاءات اضافية
    stats["total"] += 1
    stats["length_bucket_" + str(min((final_meta["orig_length"]//200)*200, 2000))] += 1

# ---------- طباعة ملخص ----------
print("Processed documents:", stats["total"])
print("Missing instruction:", stats.get("missing_instruction",0))
print("Missing response:", stats.get("missing_response",0))
print("Length buckets example counts:", {k:v for k,v in stats.items() if k.startswith("length_bucket_")})
print("\n--- Sample processed doc (first 3) ---")
for p in processed[:3]:
    print("META:", p["metadata"])
    print("TEXT (truncated):", p["text"][:400])
    print("----")


Processed documents: 937
Missing instruction: 0
Missing response: 0
Length buckets example counts: {'length_bucket_2000': 429, 'length_bucket_1400': 62, 'length_bucket_1600': 40, 'length_bucket_1000': 85, 'length_bucket_600': 78, 'length_bucket_1200': 61, 'length_bucket_800': 82, 'length_bucket_1800': 48, 'length_bucket_400': 37, 'length_bucket_200': 10, 'length_bucket_0': 5}

--- Sample processed doc (first 3) ---
META: {'source': '/content/cleaned_dataset.json', 'seq_num': 1, 'orig_length': 53973, 'has_response': True}
TEXT (truncated): Instruction: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. I’ve never had counseling about any of this. Do I have too many issues to address in counseling?
Response: It is v
----
META: {'source': '/content/cleaned_dataset.json',

# Chunking

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# اختر حجم القطعة
CHUNK_SIZE = 800         # ده ممتاز لعندك لأن النصوص طويلة
CHUNK_OVERLAP = 100      # للحفاظ على ترابط بين القطع

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " "],  # الأفضل للنصوص الطويلة
)

chunked_docs = []

for p in processed:
    # p["text"]: النص اللي عملناه Instruction + Response
    splits = splitter.split_text(p["text"])

    for s in splits:
        chunked_docs.append({
            "text": s,
            "metadata": p["metadata"]
        })

print("Total chunks:", len(chunked_docs))

# نطبع 3 أمثلة عشوائية
import random
samples = random.sample(chunked_docs, 3)

print("\n--- Sample chunks ---")
for i, c in enumerate(samples):
    print(f"\nChunk {i}:")
    print("Length:", len(c["text"]))
    print("Text:", c["text"][:300], "...")  # 300 حرف بس علشان باين
    print("Metadata:", c["metadata"])

Total chunks: 5054

--- Sample chunks ---

Chunk 0:
Length: 601
Text: Response: Yes you can help! Emotional and verbal abuse can cause a negative impact in your family. It is important that you encourage your siblings to seek assistance with a therapist, and maybe help them find one, they should have school counselors available and if not our college counselor may hel ...
Metadata: {'source': '/content/cleaned_dataset.json', 'seq_num': 441, 'orig_length': 1798, 'has_response': True}

Chunk 1:
Length: 793
Text: . Consider pre-marital counseling as well if you have not already done so. It will give you and your fiance a safe place to process this further and build a solid foundation for your relationship as you move towards marriage. I hope that you and your fiance will find healing as you work through this ...
Metadata: {'source': '/content/cleaned_dataset.json', 'seq_num': 402, 'orig_length': 12344, 'has_response': True}

Chunk 2:
Length: 615
Text: . Then you would be able to tell your

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " "],
)

# افترض processed موجود (قائمة dicts {"text","metadata"})
chunked_docs_new = []
for p in processed:
    splits = splitter.split_text(p["text"])
    for s in splits:
        chunked_docs_new.append({"text": s, "metadata": p["metadata"]})

print("Old chunks:", len(chunked_docs))
print("New chunks:", len(chunked_docs_new))

# استبدل المتغيرات للاستخدام لاحقاً
chunked_docs = chunked_docs_new

Old chunks: 5054
New chunks: 7761


In [23]:
# 1_rebuild_chunks.py
# افترض المتغير processed موجود (نتج من مرحلة preprocessing)
# لو مش موجود، شغّل الكود اللي جهز processed أولاً.

chunked_docs_new = []

for p in processed:
    full_text = p["text"]
    # فصل instruction و response بصورة آمنة
    if "Instruction:" in full_text and "Response:" in full_text:
        try:
            inst = full_text.split("Instruction:",1)[1].split("Response:",1)[0].strip()
            resp = full_text.split("Response:",1)[1].strip()
        except Exception:
            continue
    else:
        # لو ما فيش الفورمات ده، اعتبر whole text كـ response
        inst = ""
        resp = full_text.strip()

    # chunk A: response-only (الأهم للـ retrieval)
    if resp:
        chunked_docs_new.append({"text": resp, "metadata": {**p["metadata"], "chunk_type":"response"}})

    # chunk B: instruction-only (ممكن يفيد أحياناً)
    if inst:
        chunked_docs_new.append({"text": inst, "metadata": {**p["metadata"], "chunk_type":"instruction"}})

    # chunk C: full (inst + resp) - optional but مفيد للتصنيف/رابطية
    chunked_docs_new.append({"text": full_text, "metadata": {**p["metadata"], "chunk_type":"inst_resp"}})

print("Created chunks:", len(chunked_docs_new))
# استبدل chunked_docs بالنسخة الجديدة للاستخدام اللاحق
chunked_docs = chunked_docs_new

Created chunks: 2811


# (embeddings + FAISS)

In [24]:
! pip install sentence-transformers faiss-cpu tqdm

In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
from tqdm.auto import tqdm
import os
import json

In [29]:
# ---------- إعدادات ----------
EMB_MODEL = "all-mpnet-base-v2"   # أو "all-MiniLM-L6-v2" لو عايز أسرع
BATCH_SIZE = 64
USE_CUDA = True  # غيّره True لو عندك GPU
INDEX_DIR = "./faiss_index_v1"
os.makedirs(INDEX_DIR, exist_ok=True)

# ---------- تحميل موديل الـ embeddings ----------
device = "cuda" if USE_CUDA else "cpu"
print("Loading embedding model on", device)
embedder = SentenceTransformer(EMB_MODEL, device=device)

# ---------- تجهيز النصوص ----------
texts = [d["text"] for d in chunked_docs]
metadatas = [d["metadata"] for d in chunked_docs]
n = len(texts)
print("Total chunks to embed:", n)

# ---------- حساب الـ embeddings على دفعات ----------
all_embs = []
for i in tqdm(range(0, n, BATCH_SIZE), desc="Embedding batches"):
    batch_texts = texts[i:i+BATCH_SIZE]
    embs = embedder.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
    all_embs.append(embs)
all_embs = np.vstack(all_embs).astype("float32")
print("Embeddings shape:", all_embs.shape)

# ---------- تطبيع (normalize) للـ cosine similarity ----------
faiss.normalize_L2(all_embs)

# ---------- بناء Index FAISS ----------
d = all_embs.shape[1]
# IndexFlatIP مع inner-product بعد normalize => يعادل cosine
index = faiss.IndexFlatIP(d)
index.add(all_embs)
print("FAISS index: n_items =", index.ntotal)

# ---------- حفظ الـ index و metadata ----------
faiss.write_index(index, os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "wb") as f:
    pickle.dump({"texts": texts, "metadatas": metadatas}, f)

# (اختياري) حفظ بعض الإحصاءات
with open(os.path.join(INDEX_DIR, "stats.json"), "w", encoding="utf-8") as f:
    json.dump({"n_chunks": n, "embed_model": EMB_MODEL, "batch_size": BATCH_SIZE}, f, ensure_ascii=False, indent=2)

print("Saved FAISS index and metadata to", INDEX_DIR)

Loading embedding model on cuda


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total chunks to embed: 2811


Embedding batches:   0%|          | 0/44 [00:00<?, ?it/s]

Embeddings shape: (2811, 768)
FAISS index: n_items = 2811
Saved FAISS index and metadata to ./faiss_index_v1


In [30]:
# pip install sentence-transformers faiss-cpu tqdm

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import os
from tqdm.auto import tqdm

EMB_MODEL = "multi-qa-mpnet-base-dot-v1"
BATCH_SIZE = 64
USE_CUDA = True   # غيّره True لو عندك GPU
INDEX_DIR = "./faiss_index_v2"
os.makedirs(INDEX_DIR, exist_ok=True)

device = "cuda" if USE_CUDA else "cpu"
embedder = SentenceTransformer(EMB_MODEL, device=device)

texts = [d["text"] for d in chunked_docs]
metadatas = [d["metadata"] for d in chunked_docs]
n = len(texts)
print("Total chunks to embed:", n)

# encode batches
all_embs = []
for i in tqdm(range(0, n, BATCH_SIZE), desc="Embedding batches"):
    batch = texts[i:i+BATCH_SIZE]
    embs = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    all_embs.append(embs)
all_embs = np.vstack(all_embs).astype("float32")
print("Embeddings shape:", all_embs.shape)

# normalize for cosine-like (use IndexFlatIP)
faiss.normalize_L2(all_embs)

d = all_embs.shape[1]
index = faiss.IndexFlatIP(d)
index.add(all_embs)
print("Index items:", index.ntotal)

# save
faiss.write_index(index, os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "wb") as f:
    pickle.dump({"texts": texts, "metadatas": metadatas}, f)
print("Saved new FAISS index to", INDEX_DIR)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total chunks to embed: 2811


Embedding batches:   0%|          | 0/44 [00:00<?, ?it/s]

Embeddings shape: (2811, 768)
Index items: 2811
Saved new FAISS index to ./faiss_index_v2


In [31]:
# 2_embed_mpnet.py
# تثبيت لو محتاج:
# pip install sentence-transformers faiss-cpu tqdm

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import os
from tqdm.auto import tqdm

EMB_MODEL = "all-mpnet-base-v2"
BATCH_SIZE = 64
USE_CUDA = True   # غيّره True لو عندك GPU وCUDA
INDEX_DIR = "./faiss_index_mpnet_v1"
os.makedirs(INDEX_DIR, exist_ok=True)

device = "cuda" if USE_CUDA else "cpu"
print("Loading embedder on", device)
embedder = SentenceTransformer(EMB_MODEL, device=device)

texts = [d["text"] for d in chunked_docs]
metadatas = [d["metadata"] for d in chunked_docs]
n = len(texts)
print("Total chunks to embed:", n)

all_embs = []
for i in tqdm(range(0, n, BATCH_SIZE), desc="Embedding batches"):
    batch = texts[i:i+BATCH_SIZE]
    embs = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    all_embs.append(embs)
all_embs = np.vstack(all_embs).astype("float32")
print("Embeddings shape:", all_embs.shape)

# normalize
faiss.normalize_L2(all_embs)

d = all_embs.shape[1]
index = faiss.IndexFlatIP(d)
index.add(all_embs)
print("Index items:", index.ntotal)

# save index + metadata
faiss.write_index(index, os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "wb") as f:
    pickle.dump({"texts": texts, "metadatas": metadatas}, f)
print("Saved new FAISS index to", INDEX_DIR)

Loading embedder on cuda
Total chunks to embed: 2811


Embedding batches:   0%|          | 0/44 [00:00<?, ?it/s]

Embeddings shape: (2811, 768)
Index items: 2811
Saved new FAISS index to ./faiss_index_mpnet_v1


In [32]:
def retrieve_candidates_responses_only(query, top_k=50):
    # استرجاع من الفايس
    initial = retrieve_mpnet(query, k=top_k)

    # فلترة بحيث نحتفظ فقط بالردود
    filtered = [x for x in initial if x['metadata'].get('chunk_type') == "response"]

    # لو قل جدًا، زوّد candidates
    if len(filtered) < 5:
        return initial  # fallback
    return filtered

In [33]:
import faiss
import pickle
import os

INDEX_DIR = "./faiss_index_v1"

def load_index(index_dir=INDEX_DIR):
    index = faiss.read_index(os.path.join(index_dir, "index.faiss"))
    with open(os.path.join(index_dir, "metadatas.pkl"), "rb") as f:
        meta = pickle.load(f)
    return index, meta["texts"], meta["metadatas"]

index, stored_texts, stored_meta = load_index()
print("Index loaded. Total items:", index.ntotal)


Index loaded. Total items: 2811


In [34]:
import faiss, pickle, os
INDEX_DIR = "./faiss_index_v2"
index = faiss.read_index(os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "rb") as f:
    meta = pickle.load(f)
stored_texts = meta["texts"]
stored_meta = meta["metadatas"]

# embedder نفس اللي استعملناه
# embedder = SentenceTransformer(EMB_MODEL, device=device)

def retrieve_new(query, k=10):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    res = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0: continue
        res.append({"score": float(score), "text": stored_texts[idx], "metadata": stored_meta[idx]})
    return res

# تجربة سريعة
res = retrieve_new("My husband left me and I'm dealing with anxiety", k=5)
for r in res:
    print(r["score"], r["text"][:250].replace("\n"," "))
    print("---")

0.4494197368621826 I started having anxiety three months ago. I'm new to having anxiety, and it's making me depressed.
---
0.4168580174446106 I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped.
---
0.4153398275375366 My doctor thinks that seeing a psychiatrist will help with my depression and anxiety. Now my anxiety is worse, what do I do? Writing on here has taken every bit of strength I have left.
---
0.41386932134628296 I feel angry, anxious, and depressed. The PTSD I suffer is from a past relationship.
---
0.4124007225036621 Seems like you've really been through the ringer without much of a break. Just reading through the details you put in the question, I hear a couple things. One is that you seem like a helper. That's not bad at all, though other people taking advantag
---


In [35]:
from sentence_transformers import SentenceTransformer
import numpy as np

# نفس الموديل اللي استخدمناه في embedding
embedder = SentenceTransformer("all-mpnet-base-v2")

def retrieve(query, k=4):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)

    scores, indices = index.search(q_emb, k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        results.append({
            "score": float(score),
            "text": stored_texts[idx],
            "metadata": stored_meta[idx]
        })
    return results

# تجربة:
query = "My husband left me and I'm dealing with anxiety"
res = retrieve(query, k=3)

for r in res:
    print("Score:", r["score"])
    print("Text:", r["text"][:300])
    print("Metadata:", r["metadata"])
    print("-" * 40)

Score: 0.4494197368621826
Text: I started having anxiety three months ago. I'm new to having anxiety, and it's making me depressed.
Metadata: {'source': '/content/cleaned_dataset.json', 'seq_num': 97, 'orig_length': 3114, 'has_response': True, 'chunk_type': 'instruction'}
----------------------------------------
Score: 0.4168580174446106
Text: I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped.
Metadata: {'source': '/content/cleaned_dataset.json', 'seq_num': 45, 'orig_length': 3594, 'has_response': True, 'chunk_type': 'instruction'}
----------------------------------------
Score: 0.4153398275375366
Text: My doctor thinks that seeing a psychiatrist will help with my depression and anxiety. Now my anxiety is worse, what do I do? Writing on here has taken every bit of strength I have left.
Metadata: {'source': '/content/cleaned_dataset.json', 'seq_num': 136, 'orig_length': 793, 'has_response': True, 'chunk_type': 'instruction'}
----------

In [36]:
# 3_retrieve_mpnet.py
import faiss, pickle, os
from sentence_transformers import SentenceTransformer
import numpy as np

INDEX_DIR = "./faiss_index_mpnet_v1"
index = faiss.read_index(os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "rb") as f:
    meta = pickle.load(f)
stored_texts = meta["texts"]
stored_meta = meta["metadatas"]

# استخدم نفس embedder من خطوة 2
embedder = SentenceTransformer("all-mpnet-base-v2", device="cuda" if False else "cpu")

def retrieve_mpnet(query, k=10):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    res = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0: continue
        res.append({"score": float(score), "text": stored_texts[idx], "metadata": stored_meta[idx]})
    return res

# تجربة
r = retrieve_mpnet("My husband left me and I'm dealing with anxiety", k=5)
for it in r:
    print(it["score"], it["metadata"].get("chunk_type"), it["text"][:220].replace("\n"," "))
    print("---")

0.7158937454223633 instruction I started having anxiety three months ago. I'm new to having anxiety, and it's making me depressed.
---
0.649259090423584 instruction I feel angry, anxious, and depressed. The PTSD I suffer is from a past relationship.
---
0.6215025186538696 instruction I've been having horrible anxiety for the last week. I can't sleep. I get a sense of doom, and it's hard to breathe. I feel like nothing I do makes it better.
---
0.6111903190612793 instruction I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped.
---
0.6088861227035522 response Seems like you've really been through the ringer without much of a break. Just reading through the details you put in the question, I hear a couple things. One is that you seem like a helper. That's not bad at all, thoug
---


# Retrival Evaluation

In [29]:
gt_pairs = []
for item in processed:
    text = item["text"]
    inst = text.split("Instruction:",1)[1].split("Response:",1)[0].strip()
    resp = text.split("Response:",1)[1].strip()
    gt_pairs.append((inst, resp))

In [38]:
retrieve(inst, k=5)

[{'score': 1.0,
  'text': 'What are some difficulties that a counselor can encounter when dealing with a client?',
  'metadata': {'source': '/content/cleaned_dataset.json',
   'seq_num': 937,
   'orig_length': 5525,
   'has_response': True,
   'chunk_type': 'instruction'}},
 {'score': 0.8068782687187195,
  'text': 'Instruction: What are some difficulties that a counselor can encounter when dealing with a client?\nResponse: Each client brings their own style they like to a coaching or counseling relationship. When I counseled and coached military I learned to relate to what style of coaching they seek. Some were better at the Drill Sergeant style of coaching, or a liberal style, integrative, humanistic, progressive, or conservative. Some can be obsessed and try and cross the line. When you are in the public eye it can be a bit much and you have to know what client is a good fit for your practice. Some don\'t see you as human but we are. We seek to do the best we can with our knowledge c

In [39]:
def evaluate_retrieval(gt_pairs, k=5, limit=200):
    """
    gt_pairs: list of (instruction, response)
    limit: عدد العينات اللي هنقيس عليها (علشان يكون أسرع)
    """
    import numpy as np
    recall_count = 0
    reciprocal_ranks = []

    samples = gt_pairs[:limit]  # لو dataset كبير
    for inst, resp in samples:
        results = retrieve(inst, k=k)
        texts = [r["text"] for r in results]

        # هل الـ response موجود في أي chunk؟
        found_rank = None
        for i, t in enumerate(texts):
            if resp[:40] in t:  # نطابق أول 40 حرف من الـ response
                found_rank = i + 1
                break

        if found_rank:
            recall_count += 1
            reciprocal_ranks.append(1 / found_rank)
        else:
            reciprocal_ranks.append(0)

    recall = recall_count / len(samples)
    mrr = sum(reciprocal_ranks) / len(samples)
    return recall, mrr

# تجربة:
recall, mrr = evaluate_retrieval(gt_pairs, k=5, limit=200)
print("Recall@5:", recall)
print("MRR:", mrr)

Recall@5: 0.94
MRR: 0.4506666666666667


In [40]:
# requirements: sentence-transformers, numpy
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm.auto import tqdm

# افتراض: لديك قائمة gt_pairs = [(inst, resp), ...]
# وافتراض: لديك دالة retrieve(query, k) اللي بترجع [{"score","text","metadata"},...]
# وافتراض: embedder هو نفس الـ SentenceTransformer المستخدم لبناء الindex أو موديل مشابه

embedder_eval = SentenceTransformer("all-mpnet-base-v2")  # أو miniLM لتجربة أسرع

def is_semantic_match(resp_text, retrieved_text, threshold=0.75):
    # حساب cosine بين embedding للـ response والـ retrieved chunk
    a = embedder_eval.encode([resp_text], convert_to_numpy=True).astype("float32")
    b = embedder_eval.encode([retrieved_text], convert_to_numpy=True).astype("float32")
    # normalize
    a = a / np.linalg.norm(a, axis=1, keepdims=True)
    b = b / np.linalg.norm(b, axis=1, keepdims=True)
    sim = float(np.dot(a, b.T)[0][0])
    return sim >= threshold, sim

def evaluate_variants(gt_pairs, ks=[1,3,5,10], sample_limit=500, semantic_threshold=0.75):
    results = {k: {"hits":0, "mrr_sum":0.0} for k in ks}
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        for k in ks:
            retrieved = retrieve(inst, k=k)  # استخدام دالتك الحالية
            texts = [r["text"] for r in retrieved]
            found_rank = None
            # أولاً: نحاول match نصي بسيط (contains)
            for i, t in enumerate(texts):
                if resp.strip()[:50] in t:  # نسخة أحسن من أول 40 حرف
                    found_rank = i+1
                    break
            # لو ما لقيناش بالـ contains -> نجرب semantic match
            if not found_rank:
                for i, t in enumerate(texts):
                    hit, sim = is_semantic_match(resp, t, threshold=semantic_threshold)
                    if hit:
                        found_rank = i+1
                        break
            # حساب metrics
            if found_rank:
                results[k]["hits"] += 1
                results[k]["mrr_sum"] += 1.0 / found_rank
            else:
                results[k]["mrr_sum"] += 0.0
    # تجميع النتائج
    summary = {}
    for k in ks:
        recall = results[k]["hits"] / n
        mrr = results[k]["mrr_sum"] / n
        summary[k] = {"recall": recall, "mrr": mrr}
    return summary

# مثال تشغيل
summary = evaluate_variants(gt_pairs, ks=[1,3,5,10], sample_limit=200, semantic_threshold=0.75)
print(summary)


  0%|          | 0/200 [00:00<?, ?it/s]

{1: {'recall': 0.165, 'mrr': 0.165}, 3: {'recall': 0.885, 'mrr': 0.43916666666666665}, 5: {'recall': 0.945, 'mrr': 0.4531666666666667}, 10: {'recall': 0.98, 'mrr': 0.4555198412698412}}


In [41]:
# pip install -U sentence-transformers
from sentence_transformers import CrossEncoder
import numpy as np
from tqdm.auto import tqdm

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")  # خفيف وسريع

def retrieve_and_rerank(query, top_k=20, rerank_k=5):
    # استخرج أولاً top_k من الفايس
    initial = retrieve(query, k=top_k)
    pair_texts = [(query, item["text"]) for item in initial]
    scores = reranker.predict(pair_texts)  # array of scores
    # رتب النتائج بحسب scores نزولاً
    ranked = sorted(zip(initial, scores), key=lambda x: x[1], reverse=True)
    reranked = [r[0] for r in ranked][:rerank_k]
    return reranked, [s for _,s in ranked][:rerank_k]

# تجربة على مثال واحد
q = "My husband left me and I'm dealing with anxiety"
reranked, rerank_scores = retrieve_and_rerank(q, top_k=20, rerank_k=5)
for item, sc in zip(reranked, rerank_scores):
    print(sc, item["text"][:300])
    print("---")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

-3.5169716 I feel angry, anxious, and depressed. The PTSD I suffer is from a past relationship.
---
-3.5580387 I started having anxiety three months ago. I'm new to having anxiety, and it's making me depressed.
---
-4.2376833 I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped.
---
-4.272862 Instruction: I have been dealing with depression and anxiety for a number of years. I have been on medication, but lately my depression has felt worse. Can counseling help?
Response: Thank you for asking this important question. I find that there are three steps to getting ready for treatment. Step 
---
-4.4951982 Instruction: I've been having horrible anxiety for the last week. I can't sleep. I get a sense of doom, and it's hard to breathe. I feel like nothing I do makes it better.
Response: That really sucks to go through. There are tons of ways to work with anxiety, both directly and indirectly that can he
---


In [42]:
# Evaluate using reranking
def evaluate_with_rerank(gt_pairs, sample_limit=200, top_k=20, rerank_k=5):
    recall_count = 0
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        candidates, scores = retrieve_and_rerank(inst, top_k=top_k, rerank_k=rerank_k)
        texts = [c["text"] for c in candidates]
        found_rank = None
        for i, t in enumerate(texts):
            if resp.strip()[:40] in t:
                found_rank = i+1
                break
        # fallback semantic match
        if not found_rank:
            for i, t in enumerate(texts):
                hit, sim = is_semantic_match(resp, t, threshold=0.75)
                if hit:
                    found_rank = i+1
                    break
        if found_rank:
            recall_count += 1
            mrr_sum += 1.0/found_rank
    return recall_count/n, mrr_sum/n

recall_rerank, mrr_rerank = evaluate_with_rerank(gt_pairs, sample_limit=200, top_k=20, rerank_k=5)
print("Recall (reranked):", recall_rerank)
print("MRR (reranked):", mrr_rerank)

  0%|          | 0/200 [00:00<?, ?it/s]

Recall (reranked): 0.985
MRR (reranked): 0.63


In [ ]:
from tqdm.auto import tqdm
import numpy as np

# افترض gt_pairs موجود (inst, resp)
def is_semantic_match(resp_text, chunk_text, threshold=0.75):
    a = embedder.encode([resp_text], convert_to_numpy=True).astype("float32")
    b = embedder.encode([chunk_text], convert_to_numpy=True).astype("float32")
    a = a / np.linalg.norm(a, axis=1, keepdims=True)
    b = b / np.linalg.norm(b, axis=1, keepdims=True)
    sim = float(np.dot(a, b.T)[0][0])
    return sim >= threshold, sim

def evaluate_new(gt_pairs, ks=[1,3,5,10], sample_limit=200, semantic_threshold=0.75):
    results = {k: {"hits":0, "mrr_sum":0.0} for k in ks}
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        for k in ks:
            retrieved = retrieve_new(inst, k=k)
            texts = [r["text"] for r in retrieved]
            found_rank = None
            # try semantic match
            for i, t in enumerate(texts):
                hit, sim = is_semantic_match(resp, t, threshold=semantic_threshold)
                if hit:
                    found_rank = i + 1
                    break
            if found_rank:
                results[k]["hits"] += 1
                results[k]["mrr_sum"] += 1.0 / found_rank
            else:
                results[k]["mrr_sum"] += 0.0
    summary = {}
    for k in ks:
        recall = results[k]["hits"] / n
        mrr = results[k]["mrr_sum"] / n
        summary[k] = {"recall": recall, "mrr": mrr}
    return summary

summary_new = evaluate_new(gt_pairs, ks=[1,3,5,10], sample_limit=200, semantic_threshold=0.75)
print(summary_new)

  0%|          | 0/200 [00:00<?, ?it/s]

{1: {'recall': 0.13, 'mrr': 0.13}, 3: {'recall': 0.5, 'mrr': 0.29583333333333345}, 5: {'recall': 0.57, 'mrr': 0.3120833333333335}, 10: {'recall': 0.685, 'mrr': 0.32768055555555575}}


In [ ]:
# pip install -U sentence-transformers
from sentence_transformers import CrossEncoder
reranker = CrossEncoder("cross-encoder/stsb-roberta-large")  # أو change to ms-marco-MiniLM-L-6-v2

def retrieve_and_rerank_new(query, top_k=20, rerank_k=5):
    initial = retrieve_new(query, k=top_k)
    pair_texts = [(query, item["text"]) for item in initial]
    scores = reranker.predict(pair_texts)  # may be slow
    ranked = sorted(zip(initial, scores), key=lambda x: x[1], reverse=True)
    reranked = [r[0] for r in ranked][:rerank_k]
    rerank_scores = [s for _, s in ranked][:rerank_k]
    return reranked, rerank_scores

# Evaluate reranked
def evaluate_with_rerank_new(gt_pairs, sample_limit=200, top_k=20, rerank_k=5):
    recall_count = 0
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        candidates, _ = retrieve_and_rerank_new(inst, top_k=top_k, rerank_k=rerank_k)
        texts = [c["text"] for c in candidates]
        found_rank = None
        for i, t in enumerate(texts):
            if resp.strip()[:40] in t:
                found_rank = i+1
                break
        if not found_rank:
            for i, t in enumerate(texts):
                hit, sim = is_semantic_match(resp, t, threshold=0.75)
                if hit:
                    found_rank = i+1
                    break
        if found_rank:
            recall_count += 1
            mrr_sum += 1.0/found_rank
    return recall_count/n, mrr_sum/n

recall_rerank_new, mrr_rerank_new = evaluate_with_rerank_new(gt_pairs, sample_limit=200, top_k=20, rerank_k=5)
print("Recall (reranked new):", recall_rerank_new)
print("MRR (reranked new):", mrr_rerank_new)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Recall (reranked new): 0.555
MRR (reranked new): 0.2698333333333335


In [ ]:
# 4_eval_mpnet.py
from tqdm.auto import tqdm
import numpy as np

# افتراض: gt_pairs موجود (inst, resp) كما قبل
# نستخدم embedder (mpnet) لمطابقة الresponse مع كل chunk باستخدام cosine

def is_semantic_match_mpnet(resp_text, chunk_text, threshold=0.75):
    a = embedder.encode([resp_text], convert_to_numpy=True).astype("float32")
    b = embedder.encode([chunk_text], convert_to_numpy=True).astype("float32")
    a = a / np.linalg.norm(a, axis=1, keepdims=True)
    b = b / np.linalg.norm(b, axis=1, keepdims=True)
    sim = float(np.dot(a, b.T)[0][0])
    return sim >= threshold, sim

def evaluate_mpnet(gt_pairs, ks=[1,3,5,10], sample_limit=200, semantic_threshold=0.75):
    results = {k: {"hits":0, "mrr_sum":0.0} for k in ks}
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        for k in ks:
            retrieved = retrieve_mpnet(inst, k=k)
            texts = [r["text"] for r in retrieved]
            found_rank = None
            for i, t in enumerate(texts):
                hit, sim = is_semantic_match_mpnet(resp, t, threshold=semantic_threshold)
                if hit:
                    found_rank = i + 1
                    break
            if found_rank:
                results[k]["hits"] += 1
                results[k]["mrr_sum"] += 1.0 / found_rank
            else:
                results[k]["mrr_sum"] += 0.0
    summary = {}
    for k in ks:
        recall = results[k]["hits"] / n
        mrr = results[k]["mrr_sum"] / n
        summary[k] = {"recall": recall, "mrr": mrr}
    return summary

summary_mpnet = evaluate_mpnet(gt_pairs, ks=[1,3,5,10], sample_limit=200, semantic_threshold=0.75)
print(summary_mpnet)

  0%|          | 0/200 [00:00<?, ?it/s]

{1: {'recall': 0.165, 'mrr': 0.165}, 3: {'recall': 0.865, 'mrr': 0.51}, 5: {'recall': 0.925, 'mrr': 0.524}, 10: {'recall': 0.96, 'mrr': 0.5283531746031744}}


In [ ]:
# pip install -U sentence-transformers tqdm

from sentence_transformers import CrossEncoder
from tqdm.auto import tqdm
import numpy as np

# لو حجم العيّنة كبير خفّف النموذج مؤقتًا أو اختبر على sample_limit صغير أولاً
reranker = CrossEncoder("cross-encoder/stsb-roberta-large")  # أو ms-marco-MiniLM-L-6-v2 لو عايز أسرع

def retrieve_and_rerank_mpnet(query, top_k=20, rerank_k=5):
    initial = retrieve_mpnet(query, k=top_k)  # استخدم دالتك الحالية
    pair_texts = [(query, item["text"]) for item in initial]
    scores = reranker.predict(pair_texts, show_progress_bar=False)  # قد يكون بطيئًا
    ranked = sorted(zip(initial, scores), key=lambda x: x[1], reverse=True)
    reranked = [r[0] for r in ranked][:rerank_k]
    return reranked, [s for _,s in ranked][:rerank_k]

def evaluate_with_rerank(gt_pairs, sample_limit=200, top_k=20, rerank_k=5):
    recall_counts = {1:0,3:0,5:0,10:0}
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        candidates, _ = retrieve_and_rerank_mpnet(inst, top_k=top_k, rerank_k=rerank_k)
        texts = [c["text"] for c in candidates]
        found_rank = None
        for i, t in enumerate(texts):
            if resp.strip()[:40] in t:  # سريع؛ لو عايز استبدل بـ semantic match استخدم embedder
                found_rank = i+1
                break
        # fallback semantic match
        if not found_rank:
            a = embedder.encode([resp], convert_to_numpy=True).astype("float32")
            for i, t in enumerate(texts):
                b = embedder.encode([t], convert_to_numpy=True).astype("float32")
                a_norm = a / np.linalg.norm(a, axis=1, keepdims=True)
                b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
                sim = float(np.dot(a_norm, b_norm.T)[0][0])
                if sim >= 0.75:
                    found_rank = i+1
                    break
        if found_rank:
            mrr_sum += 1.0 / found_rank
            for k in (1,3,5,10):
                if found_rank <= k:
                    recall_counts[k] += 1
    summary = {}
    for k in (1,3,5,10):
        summary[k] = {"recall": recall_counts[k]/n}
    summary["mrr"] = mrr_sum / n
    return summary

# Example run (جرب sample_limit=100 أولاً لو البطء مشكلة)
res = evaluate_with_rerank(gt_pairs, sample_limit=100, top_k=20, rerank_k=5)
print(res)

  0%|          | 0/100 [00:00<?, ?it/s]

{1: {'recall': 0.02}, 3: {'recall': 0.93}, 5: {'recall': 0.98}, 10: {'recall': 0.98}, 'mrr': 0.48033333333333333}


In [ ]:
# pip install -U sentence-transformers tqdm

from sentence_transformers import CrossEncoder
from tqdm.auto import tqdm
import numpy as np

reranker = CrossEncoder("cross-encoder/ms-marco-TinyBERT-L-2-v2")

def retrieve_and_rerank_mpnet(query, top_k=20, rerank_k=5):
    initial = retrieve_mpnet(query, k=top_k)  # استخدم دالتك الحالية
    pair_texts = [(query, item["text"]) for item in initial]
    scores = reranker.predict(pair_texts, show_progress_bar=False)  # قد يكون بطيئًا
    ranked = sorted(zip(initial, scores), key=lambda x: x[1], reverse=True)
    reranked = [r[0] for r in ranked][:rerank_k]
    return reranked, [s for _,s in ranked][:rerank_k]

def evaluate_with_rerank(gt_pairs, sample_limit=200, top_k=20, rerank_k=5):
    recall_counts = {1:0,3:0,5:0,10:0}
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        candidates, _ = retrieve_and_rerank_mpnet(inst, top_k=top_k, rerank_k=rerank_k)
        texts = [c["text"] for c in candidates]
        found_rank = None
        for i, t in enumerate(texts):
            if resp.strip()[:40] in t:  # سريع؛ لو عايز استبدل بـ semantic match استخدم embedder
                found_rank = i+1
                break
        # fallback semantic match
        if not found_rank:
            a = embedder.encode([resp], convert_to_numpy=True).astype("float32")
            for i, t in enumerate(texts):
                b = embedder.encode([t], convert_to_numpy=True).astype("float32")
                a_norm = a / np.linalg.norm(a, axis=1, keepdims=True)
                b_norm = b / np.linalg.norm(b, axis=1, keepdims=True)
                sim = float(np.dot(a_norm, b_norm.T)[0][0])
                if sim >= 0.75:
                    found_rank = i+1
                    break
        if found_rank:
            mrr_sum += 1.0 / found_rank
            for k in (1,3,5,10):
                if found_rank <= k:
                    recall_counts[k] += 1
    summary = {}
    for k in (1,3,5,10):
        summary[k] = {"recall": recall_counts[k]/n}
    summary["mrr"] = mrr_sum / n
    return summary

# Example run (جرب sample_limit=100 أولاً لو البطء مشكلة)
res = evaluate_with_rerank(gt_pairs, sample_limit=100, top_k=20, rerank_k=5)
print(res)

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{1: {'recall': 0.04}, 3: {'recall': 0.99}, 5: {'recall': 0.99}, 10: {'recall': 0.99}, 'mrr': 0.515}


In [ ]:
def rerank_responses(query, candidates, rerank_k=5):
    pair_texts = [(query, item["text"]) for item in candidates]
    scores = reranker.predict(pair_texts)
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return ranked[:rerank_k]

In [ ]:
# طبع أول 10 درجات من reranker على مثال واحد
query = gt_pairs[0][0]   # مثال: أول instruction
initial = retrieve_mpnet(query, k=20)
pair_texts = [(query, item["text"]) for item in initial]

scores = reranker.predict(pair_texts)
for i, (it, sc) in enumerate(zip(initial, scores)):
    print(i+1, "score:", sc, "chunk_type:", it['metadata'].get('chunk_type'))
    print(it['text'][:200].replace("\n"," "))
    print("-"*40)

1 score: 0.9469649 chunk_type: instruction
I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I
----------------------------------------
2 score: 0.6903365 chunk_type: inst_resp
Instruction: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to ha
----------------------------------------
3 score: 0.48275447 chunk_type: response
It is very common for people to have multiple issues that they want to (and need to) address in counseling. I have had clients ask that same question and through more exploration, there is often an un
----------------------------------------
4 score: 0.2728651 chunk_type: instruction
I have several issues that I need to work through. However, I am afraid of and do not trust therapists aft

In [ ]:
from collections import defaultdict
fail_examples = []
N = 50  # افحص 50 حالة أولاً

for inst, resp in gt_pairs[:N]:
    initial = retrieve_mpnet(inst, k=20)
    texts_init = [t['text'] for t in initial]
    # رتّب بالscore الFAISS (اللي هو ترتيب افتراضي)
    # ابحث مكان الـ response الأصلي في initial (بـ substring أو semantic match)
    found_init_idx = None
    for i,t in enumerate(texts_init):
        if resp.strip()[:40] in t:
            found_init_idx = i+1
            break
    # rerank
    pair_texts = [(inst, item["text"]) for item in initial]
    scores = reranker.predict(pair_texts)
    ranked = sorted(zip(initial, scores), key=lambda x: x[1], reverse=True)  # او reverse=False لو لازم
    texts_reranked = [r[0]['text'] for r in ranked]
    found_rerank_idx = None
    for i,t in enumerate(texts_reranked):
        if resp.strip()[:40] in t:
            found_rerank_idx = i+1
            break
    if found_init_idx and (found_rerank_idx is None or found_rerank_idx > found_init_idx):
        fail_examples.append({
            "inst": inst,
            "resp": resp[:200],
            "init_rank": found_init_idx,
            "rerank_rank": found_rerank_idx,
            "initial_top3": texts_init[:3],
            "reranked_top3": texts_reranked[:3],
            "scores_top3": [s for _,s in ranked[:3]]
        })

# طبع أول 5 فشل أمثلة
for e in fail_examples[:5]:
    print("Query:", e['inst'][:120])
    print("Resp preview:", e['resp'])
    print("init_rank:", e['init_rank'], "rerank_rank:", e['rerank_rank'])
    print("initial top3:")
    for t in e['initial_top3']: print("-", t[:140].replace("\n"," "))
    print("reranked top3:")
    for t in e['reranked_top3']: print("-", t[:140].replace("\n"," "))
    print("scores:", e['scores_top3'])
    print("="*80)

Query: I'm very depressed. How do I find someone to talk to?
Resp preview: Silly you took the first step to "claim it" you are by posting on here. All of us are here to help. It is to your credit that you reached out for help on this platform. How to find someone to talk to 
init_rank: 4 rerank_rank: 7
initial top3:
- I'm very depressed. How do I find someone to talk to?
- I'm in a state of depression right now. Who can I talk to? I've been sick in a lot of pain and crying. Don't know where to turn.
- I am broke, but I am sure I have been depressed for the past couple of years. I have always had anxiety. I just need someone to talk to righ
reranked top3:
- I'm very depressed. How do I find someone to talk to?
- I'm in a state of depression right now. Who can I talk to? I've been sick in a lot of pain and crying. Don't know where to turn.
- Instruction: I am not sure if I am depressed. I don't know how to bring it up to my parents, and that makes me miserable. Response: You are 
scores:

In [ ]:
{
    "instruction": "My husband told me last year he never loved me and should have never married me. He's been talking about divorce, but he hasn't filed. Now he has a girlfriend and he abandoned me and his kids. He has no financial help, and he has been out of the home for four months. He became very angry and mean. He says it's all my fault, and that we're not compatible.",
    "response": "It's hard to say that it is a midlife crisis stage.. could be... but so much more seems to be going on or have been going on to say with confidence that it is not much more."
  },

# test

In [43]:
# A_load_index_and_retrieve.py
import faiss, pickle, os
from sentence_transformers import SentenceTransformer
import numpy as np

# --- إعدادات ---
INDEX_DIR = "./faiss_index_mpnet_v1"   # غير لو ملفك تاني
EMB_MODEL = "all-mpnet-base-v2"        # تأكد إنه نفس اللي استخدمته عند البنية
DEVICE = "cuda"                         # غيّر إلى "cuda" لو عندك GPU

# --- تحميل الـ index والـ metadata ---
index = faiss.read_index(os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "rb") as f:
    meta = pickle.load(f)
stored_texts = meta["texts"]
stored_meta = meta["metadatas"]

print("Loaded FAISS index. Items:", index.ntotal)

# --- تحميل embedder للـ query embedding (نفس موديل MPNet) ---
embedder = SentenceTransformer(EMB_MODEL, device=DEVICE)

# --- دالة retrieve الأساسية (ترجع top_k من FAISS) ---
def retrieve_mpnet_raw(query, k=50):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0: continue
        results.append({
            "score": float(score),
            "text": stored_texts[idx],
            "metadata": stored_meta[idx]
        })
    return results

# --- اختبار سريع ---
res = retrieve_mpnet_raw("I'm very depressed. How do I find someone to talk to?", k=10)
for i,r in enumerate(res[:5]):
    print(i+1, "score:", r["score"], "chunk_type:", r["metadata"].get("chunk_type"))
    print(r["text"][:200].replace("\n"," "))
    print("-"*40)

Loaded FAISS index. Items: 2811
1 score: 1.0 chunk_type: instruction
I'm very depressed. How do I find someone to talk to?
----------------------------------------
2 score: 0.8047124743461609 chunk_type: instruction
I'm in a state of depression right now. Who can I talk to? I've been sick in a lot of pain and crying. Don't know where to turn.
----------------------------------------
3 score: 0.7926219701766968 chunk_type: instruction
I am broke, but I am sure I have been depressed for the past couple of years. I have always had anxiety. I just need someone to talk to right now. I don't have any friends or family I can talk to. I'm
----------------------------------------
4 score: 0.7260887026786804 chunk_type: inst_resp
Instruction: I'm very depressed. How do I find someone to talk to? Response: Silly you took the first step to "claim it" you are by posting on here. All of us are here to help. It is to your credit th
----------------------------------------
5 score: 0.7058603167533875 

In [44]:
# B_filter_responses_only.py

def filter_responses_only(candidates, min_candidates=5):
    """
    candidates: list returned from retrieve_mpnet_raw
    if after filtering we have too few items, fallback لإرجاع الـ candidates الأصلية.
    """
    filtered = [c for c in candidates if c["metadata"].get("chunk_type") == "response"]
    if len(filtered) < min_candidates:
        # fallback: لو مفيش ردود كفاية، اديهم الـ original (عشان ما يبقاش فاضي)
        return candidates
    return filtered

# --- اختبار الفلتر ---
cands = retrieve_mpnet_raw("I'm very depressed. How do I find someone to talk to?", k=50)
filtered = filter_responses_only(cands, min_candidates=5)
print("Candidates before:", len(cands), "after filter:", len(filtered))
for i,c in enumerate(filtered[:3]):
    print(i+1, c["metadata"].get("chunk_type"), c["text"][:160].replace("\n"," "))
    print("-"*30)


Candidates before: 50 after filter: 11
1 response Silly you took the first step to "claim it" you are by posting on here. All of us are here to help. It is to your credit that you reached out for help on this p
------------------------------
2 response Love yourself more and treat yourself kindly, with empathy and compassion. Usually when people feel depressed they also feel lonely and isolated. Since at this 
------------------------------
3 response First of all, if you’re sick and in pain, have you seen your doctor? If you have a chronic illness and pain, this can often lead to depression and is very commo
------------------------------


In [45]:
# C_load_reranker_and_rerank.py
from sentence_transformers import CrossEncoder

# --- اختار موديل MS-MARCO مناسب ---
RERANK_MODEL = "cross-encoder/ms-marco-TinyBERT-L-2-v2"  # خفيف ودقيق للترتيب
# البدائل: "cross-encoder/ms-marco-MiniLM-L-6-v2" أو "cross-encoder/ms-marco-electra-base"

print("Loading cross-encoder:", RERANK_MODEL)
reranker = CrossEncoder(RERANK_MODEL, device=DEVICE)  # device نفس قيمة الجزء A

def rerank_responses(query, candidates, rerank_k=5, batch_size=64):
    """
    candidates: list of dicts with 'text'
    returns: list of (candidate, score) sorted desc
    """
    pair_texts = [(query, c["text"]) for c in candidates]
    # قد نحتاج لتقسيم لو length كبير:
    scores = reranker.predict(pair_texts, show_progress_bar=False)
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return ranked[:rerank_k]

# --- اختبار rerank على استعلام ---
cands = retrieve_mpnet_raw("I'm very depressed. How do I find someone to talk to?", k=50)
cands = filter_responses_only(cands, min_candidates=5)
ranked = rerank_responses("I'm very depressed. How do I find someone to talk to?", cands, rerank_k=5)
for i,(item, sc) in enumerate(ranked):
    print(i+1, "score:", sc)
    print(item["text"][:220].replace("\n"," "))
    print("meta:", item["metadata"].get("chunk_type"))
    print("-"*60)


Loading cross-encoder: cross-encoder/ms-marco-TinyBERT-L-2-v2


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

1 score: 0.33594537
First of all, if you’re sick and in pain, have you seen your doctor? If you have a chronic illness and pain, this can often lead to depression and is very common. You’ve taken the first step to dealing with it. You can t
meta: response
------------------------------------------------------------
2 score: -0.51903063
Love yourself more and treat yourself kindly, with empathy and compassion. Usually when people feel depressed they also feel lonely and isolated. Since at this time you wouldn't want to tell anyone about the way you feel
meta: response
------------------------------------------------------------
3 score: -2.5858128
You are not alone, many people fear opening up to family members about the topic of depression or mental illness. There are many different reason why some may fear telling their parents. The most common thoughts I hear i
meta: response
------------------------------------------------------------
4 score: -2.886294
Feeling alone and/or isolated

In [46]:
# D_answer_pipeline.py

def retrieve_and_rerank_pipeline(query, top_k=50, rerank_k=5):
    initial = retrieve_mpnet_raw(query, k=top_k)
    candidates = filter_responses_only(initial, min_candidates=5)
    ranked = rerank_responses(query, candidates, rerank_k=rerank_k)
    # رجّع شكل سهل للطباعة
    return [{"score": float(s), "text": c["text"], "metadata": c["metadata"]} for c,s in ranked]

# --- تجربة على استعلامات يدوية ---
queries = [
    "I'm very depressed. How do I find someone to talk to?",
    "My husband left me and I'm dealing with anxiety",
    "How can I get to a place where I can be content from day to day?"
]

for q in queries:
    print("=== QUERY:", q)
    out = retrieve_and_rerank_pipeline(q, top_k=50, rerank_k=5)
    for i, r in enumerate(out):
        print(i+1, "score:", r["score"], "type:", r["metadata"].get("chunk_type"))
        print(r["text"][:300].replace("\n"," "))
        print("-"*50)
    print("\n")


=== QUERY: I'm very depressed. How do I find someone to talk to?
1 score: 0.33594536781311035 type: response
First of all, if you’re sick and in pain, have you seen your doctor? If you have a chronic illness and pain, this can often lead to depression and is very common. You’ve taken the first step to dealing with it. You can talk to your doctor, even if your depression isn’t related to your health. He or 
--------------------------------------------------
2 score: -0.5190306305885315 type: response
Love yourself more and treat yourself kindly, with empathy and compassion. Usually when people feel depressed they also feel lonely and isolated. Since at this time you wouldn't want to tell anyone about the way you feel, which includes the possibility that a few of the ones whom you tell would offe
--------------------------------------------------
3 score: -2.58581280708313 type: response
You are not alone, many people fear opening up to family members about the topic of depression or men

In [47]:
# E_evaluate_pipeline.py
from tqdm.auto import tqdm
import numpy as np

def evaluate_pipeline(gt_pairs, sample_limit=200, top_k=50, rerank_k=5):
    recall_counts = {1:0,3:0,5:0,10:0}
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        ranked = retrieve_and_rerank_pipeline(inst, top_k=top_k, rerank_k=rerank_k)
        texts = [r["text"] for r in ranked]
        found_rank = None
        # محاولة مطابقة بسيطة أولاً (contains)
        for i,t in enumerate(texts):
            if resp.strip()[:40] in t:
                found_rank = i+1
                break
        # لو مش موجود، استخدام embedding semantic check (mpnet)
        if not found_rank:
            a = embedder.encode([resp], convert_to_numpy=True).astype("float32")
            a = a / np.linalg.norm(a, axis=1, keepdims=True)
            for i,t in enumerate(texts):
                b = embedder.encode([t], convert_to_numpy=True).astype("float32")
                b = b / np.linalg.norm(b, axis=1, keepdims=True)
                sim = float(np.dot(a, b.T)[0][0])
                if sim >= 0.75:
                    found_rank = i+1
                    break
        if found_rank:
            mrr_sum += 1.0 / found_rank
            for k in (1,3,5,10):
                if found_rank <= k:
                    recall_counts[k] += 1
    summary = {k: recall_counts[k]/n for k in (1,3,5,10)}
    summary["mrr"] = mrr_sum / n
    return summary

# --- تشغيل التقييم (ابتداءً بعينة صغيرة لو الأول) ---
summary_after = evaluate_pipeline(gt_pairs, sample_limit=200, top_k=50, rerank_k=5)
print("Evaluation after response-only rerank:", summary_after)

  0%|          | 0/200 [00:00<?, ?it/s]

Evaluation after response-only rerank: {1: 0.49, 3: 0.665, 5: 0.805, 10: 0.805, 'mrr': 0.5991666666666666}


# last test

In [26]:
# A_build_pair_chunks.py
# تشغيل هذا الجزء أولًا

pair_chunks = []

# افترض "processed" موجود (نتج من الـ preprocessing السابق)
for p in processed:
    full_text = p["text"]
    # نحاول نفصل Instruction و Response بأمان
    if "Instruction:" in full_text and "Response:" in full_text:
        try:
            inst = full_text.split("Instruction:",1)[1].split("Response:",1)[0].strip()
            resp = full_text.split("Response:",1)[1].strip()
        except Exception:
            # لو حصل خطأ في الفصل، خليه كله كـ response
            inst = ""
            resp = full_text.strip()
    else:
        # لو الفورمات غير متوفر اعتبر النص كله response
        inst = ""
        resp = full_text.strip()

    # نص الزوج الموحد — التنسيق مهم لكنه بسيط
    pair_text = f"Instruction: {inst}\nResponse: {resp}"

    # metadata احتياطي ـ مثلاً نحتفظ ب seq_num / source
    meta = dict(p.get("metadata", {}))
    meta.update({"pair_type":"inst_resp_pair"})

    pair_chunks.append({
        "text": pair_text,
        "metadata": meta
    })

print("Created pair chunks:", len(pair_chunks))
# لو حابب خزّنهم كملف json للمرجع:
import json
with open("pair_chunks_preview.json", "w", encoding="utf-8") as f:
    json.dump([{"text": c["text"][:1000], "metadata": c["metadata"]} for c in pair_chunks[:200]], f, ensure_ascii=False, indent=2)

# الآن انتهى الجزء (A). انتقل للجزء (B).

Created pair chunks: 937


In [27]:
# B_embed_and_build_index.py
# تشغيل بعد A_build_pair_chunks.py

# Requirements:
# pip install sentence-transformers faiss-cpu tqdm

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import os
from tqdm.auto import tqdm

# --- إعدادات ---
PAIR_CHUNKS = pair_chunks  # الناتج من الجزء A
EMB_MODEL = "all-mpnet-base-v2"   # جودة عالية
#EMB_MODEL = "all-MiniLM-L6-v2"  # بديل أسرع
BATCH_SIZE = 64
USE_CUDA = True  # غيّر True لو عندك GPU
INDEX_DIR = "./faiss_index_pair_v1"
os.makedirs(INDEX_DIR, exist_ok=True)

device = "cuda" if USE_CUDA else "cpu"
print("Loading embedder:", EMB_MODEL, "on", device)
embedder = SentenceTransformer(EMB_MODEL, device=device)

texts = [c["text"] for c in PAIR_CHUNKS]
metadatas = [c["metadata"] for c in PAIR_CHUNKS]
n = len(texts)
print("Total pairs to embed:", n)

# --- حساب embeddings دفعات ---
all_embs = []
for i in tqdm(range(0, n, BATCH_SIZE), desc="Embedding batches"):
    batch = texts[i:i+BATCH_SIZE]
    embs = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False)
    all_embs.append(embs)
all_embs = np.vstack(all_embs).astype("float32")
print("Embeddings shape:", all_embs.shape)

# --- تطبيع لتمثيل cosine عبر inner-product ---
faiss.normalize_L2(all_embs)

# --- إنشاء index (IndexFlatIP) ---
d = all_embs.shape[1]
index = faiss.IndexFlatIP(d)
index.add(all_embs)
print("FAISS index built, total vectors:", index.ntotal)

# --- حفظ index وميتاداتا ---
faiss.write_index(index, os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "wb") as f:
    pickle.dump({"texts": texts, "metadatas": metadatas}, f)
print("Saved index and metadata to", INDEX_DIR)

# انتهى الجزء (B). انتقل للجزء (C).

Loading embedder: all-mpnet-base-v2 on cuda


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total pairs to embed: 937


Embedding batches:   0%|          | 0/15 [00:00<?, ?it/s]

Embeddings shape: (937, 768)
FAISS index built, total vectors: 937
Saved index and metadata to ./faiss_index_pair_v1


In [30]:
# C_query_and_evaluate.py
# تشغيل بعد B_embed_and_build_index.py

import faiss, pickle, os
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

INDEX_DIR = "./faiss_index_pair_v1"
EMB_MODEL = "all-mpnet-base-v2"  # نفس الموديل اللي استخدمته في (B)
DEVICE = "cpu"  # غيّره إلى "cuda" لو فعلت USE_CUDA=True

# --- تحميل index و metadata ---
index = faiss.read_index(os.path.join(INDEX_DIR, "index.faiss"))
with open(os.path.join(INDEX_DIR, "metadatas.pkl"), "rb") as f:
    meta = pickle.load(f)
stored_texts = meta["texts"]
stored_meta = meta["metadatas"]
print("Loaded index. Items:", index.ntotal)

# --- embedder للسؤال/الاستعلام ---
embedder_q = SentenceTransformer(EMB_MODEL, device=DEVICE)

def query_pair_index(query, k=5):
    q_emb = embedder_q.encode([query], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0: continue
        results.append({
            "score": float(score),
            "text": stored_texts[idx],
            "metadata": stored_meta[idx]
        })
    return results

# --- اختبار سريع ---
q = "I'm very depressed. How do I find someone to talk to?"
res = query_pair_index(q, k=5)
for i,r in enumerate(res):
    print(i+1, "score:", r["score"])
    print(r["text"][:400].replace("\n"," "))
    print("-"*40)

# --- دوال تقييم Recall@k و MRR ---
# افتراض: gt_pairs = [(instruction, response), ...] متوفر من قبل
def evaluate_pair_index(gt_pairs, ks=(1,3,5,10), sample_limit=200):
    recall_counts = {k:0 for k in ks}
    mrr_sum = 0.0
    n = min(len(gt_pairs), sample_limit)
    for inst, resp in tqdm(gt_pairs[:n]):
        ranked = query_pair_index(inst, k=max(ks))
        found_rank = None
        # match: نبحث إن الـ response موجود داخل نص الـ pair result
        for i, r in enumerate(ranked):
            # نتحقق إن response جزء من النص المسترجع (أو أطروحة أخرى يمكن تعديلها)
            if resp.strip()[:80] in r["text"]:
                found_rank = i+1
                break
        # fallback: تعيين similarity بالـ embedding (لو contains فشل)
        if not found_rank:
            # احتساب similarity بين resp و كل نص مسترجع
            a = embedder_q.encode([resp], convert_to_numpy=True).astype("float32")
            a = a / np.linalg.norm(a, axis=1, keepdims=True)
            for i,r in enumerate(ranked):
                b = embedder_q.encode([r["text"]], convert_to_numpy=True).astype("float32")
                b = b / np.linalg.norm(b, axis=1, keepdims=True)
                sim = float(np.dot(a, b.T)[0][0])
                if sim >= 0.75:
                    found_rank = i+1
                    break
        if found_rank:
            mrr_sum += 1.0 / found_rank
            for k in ks:
                if found_rank <= k:
                    recall_counts[k] += 1
    summary = {k: recall_counts[k]/n for k in ks}
    summary["mrr"] = mrr_sum / n
    return summary

# --- تشغيل التقييم (ابدأ بعينة صغيرة أولاً) ---
summary_pair = evaluate_pair_index(gt_pairs, ks=(1,3,5,10), sample_limit=200)
print("Pair-index evaluation:", summary_pair)

Loaded index. Items: 937
1 score: 0.7260887622833252
Instruction: I'm very depressed. How do I find someone to talk to? Response: Silly you took the first step to "claim it" you are by posting on here. All of us are here to help. It is to your credit that you reached out for help on this platform. How to find someone to talk to would depend upon your lifestyle and worldview. Many people reach out to their faith community and others to friends, famil
----------------------------------------
2 score: 0.7058603167533875
Instruction: I am broke, but I am sure I have been depressed for the past couple of years. I have always had anxiety. I just need someone to talk to right now. I don't have any friends or family I can talk to. I'm on the verge of just giving up. Response: Good question. There are resources out there - people to talk to. I'd try going to this link and calling the free 24/7 hotline:
----------------------------------------
3 score: 0.6744893789291382
Instruction: I'm in a st

  0%|          | 0/200 [00:00<?, ?it/s]

Pair-index evaluation: {1: 0.935, 3: 0.98, 5: 0.985, 10: 0.985, 'mrr': 0.9551666666666668}


# Load The Model

In [ ]:
!pip install -q unsloth
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 22.7 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# اختار موديل من موديلات Unsloth على HF
MODEL_ID = "unsloth/Llama-3.2-3B-Instruct"  # غيّر للي تحبه من القائمة

max_seq_length = 2048  # كفاية للأسئلة والإجابات
dtype = None           # يختار الأنسب (fp16/bf16) حسب الGPU
load_in_4bit = True    # علشان يوفر VRAM في Colab

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)  # مهم علشان inference سريع

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNo

In [ ]:
import torch

# تحديث دالة التوليد: نضمن إرسال attention_mask ونقصّ النتيجة بحيث نرجع بس الجزء المولَّد (مش prompt)
def generate_answer(question: str, max_new_tokens: int = 256, temperature: float = 0.2):
    prompt = (
        "You are a helpful, empathetic mental health assistant. "
        "You give supportive, safe, non-judgmental answers and encourage seeking professional help when needed.\n\n"
        f"User: {question}\nAssistant:"
    )

    # tokenization مع attention_mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # توليد مع تمرير attention_mask صراحة
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode: نص كامل = prompt + generated؛ نريد الجزء الجديد فقط
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)

    # لو عايز الجزء اللي بعد الـ prompt بس:
    # نحسب طول الـ prompt tokens
    prompt_len = input_ids.shape[1]
    generated_tokens = out[0][prompt_len:]
    generated_only = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return generated_only if generated_only else generated_text

In [ ]:
import json, time, traceback, os

sample_n = 5
results = []
path = "sample_comparison.json"

for idx, item in enumerate(cleaned_dataset[:sample_n]):
    qid = item.get("questionID") if item.get("questionID") is not None else f"idx_{idx}"
    question = item.get("instruction") or item.get("questionTitle") or ""
    gt = item.get("response")

    try:
        print(f"\nGenerating for QID = {qid} ({idx+1}/{sample_n})")
        pred = generate_answer(question, max_new_tokens=300, temperature=0.2)
    except Exception as e:
        print("Error during generation for QID", qid, ":", e)
        traceback.print_exc()
        pred = f"[ERROR_GENERATING] {e}"

    results.append({
        "questionID": qid,
        "instruction": question,
        "ground_truth": gt,
        "model_prediction": pred
    })

    # طباعة طول results للتأكد
    print("Length of results so far:", len(results))

    # حفظ آمن (اكتب لملف مؤقت ثم ابدله)
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, path)  # atomic replace
    print("Saved", len(results), "records to", path)

    time.sleep(0.4)


Generating for QID = idx_0 (1/5)
Length of results so far: 1
Saved 1 records to sample_comparison.json

Generating for QID = idx_1 (2/5)
Length of results so far: 2
Saved 2 records to sample_comparison.json

Generating for QID = idx_2 (3/5)
Length of results so far: 3
Saved 3 records to sample_comparison.json

Generating for QID = idx_3 (4/5)
Length of results so far: 4
Saved 4 records to sample_comparison.json

Generating for QID = idx_4 (5/5)
Length of results so far: 5
Saved 5 records to sample_comparison.json


In [ ]:
import json
with open("sample_comparison.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print("Final file contains", len(data), "records")

Final file contains 5 records


In [ ]:
# ====== Load file (JSON or JSONL) and compute metrics for the saved samples ======
import json, os
from pathlib import Path

# حاول يقرأ JSONL أولا، بعدين JSON
if Path("sample_comparison.jsonl").exists():
    data = []
    with open("sample_comparison.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
elif Path("sample_comparison.json").exists():
    with open("sample_comparison.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    raise FileNotFoundError("sample_comparison.json(.l) غير موجود. اتأكد من اسم الملف.")

print("Loaded", len(data), "records.")

Loaded 5 records.


In [ ]:
# ====== Install libs if لازم (فقط لو مش منصبة) ======
!pip install -q sacrebleu rouge-score bert-score sentence-transformers numpy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# ====== Imports للتقييم ======
import numpy as np
import sacrebleu
from rouge_score import rouge_scorer
import bert_score
from sentence_transformers import SentenceTransformer, util

In [ ]:
# ====== إعدادات ======
rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # سريع ومتعدد اللغات جيد

refs = [rec['ground_truth'] for rec in data]
preds = [rec['model_prediction'] for rec in data]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ====== BERTScore دفعة واحدة ======
P, R, F = bert_score.score(preds, refs, lang='en', verbose=False)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ====== Embeddings cosine دفعة واحدة ======
embs_preds = embedder.encode(preds, convert_to_tensor=True, show_progress_bar=False)
embs_refs = embedder.encode(refs, convert_to_tensor=True, show_progress_bar=False)
cosine_batch = util.cos_sim(embs_preds, embs_refs).diagonal().cpu().numpy()

In [ ]:
# ====== حساب المقاييس لكل مثال وتجميع تقرير ======
results = []
for i, rec in enumerate(data):
    ref = refs[i]
    pred = preds[i]
    bleu = sacrebleu.sentence_bleu(pred, [ref]).score if ref and pred else 0.0
    r = rouge.score(ref, pred)
    bert_f = float(F[i].cpu().item())
    cosine = float(cosine_batch[i])
    results.append({
        "questionID": rec.get("questionID"),
        "bleu": bleu,
        "rouge1": r['rouge1'].fmeasure,
        "rouge2": r['rouge2'].fmeasure,
        "rougeL": r['rougeL'].fmeasure,
        "bert_f": bert_f,
        "cosine": cosine
    })

In [ ]:
# ====== ملخص إحصائي ======
def mean(xs): return float(np.mean(xs)) if xs else 0.0
report = {
    "n": len(results),
    "bleu_mean": mean([r['bleu'] for r in results]),
    "rouge1_mean": mean([r['rouge1'] for r in results]),
    "rouge2_mean": mean([r['rouge2'] for r in results]),
    "rougeL_mean": mean([r['rougeL'] for r in results]),
    "bert_f_mean": mean([r['bert_f'] for r in results]),
    "cosine_mean": mean([r['cosine'] for r in results])
}

print("\n=== Summary ===")
for k,v in report.items():
    print(f"{k}: {v}")


=== Summary ===
n: 5
bleu_mean: 1.657020717581025
rouge1_mean: 0.29301722975551786
rouge2_mean: 0.06901644377711276
rougeL_mean: 0.1254957485308378
bert_f_mean: 0.8354787349700927
cosine_mean: 0.6913317561149597


In [ ]:
# ====== احفظ النتائج مع المقاييس في ملف منفصل ======
out_path = "sample_comparison_with_metrics.json"
for i, rec in enumerate(data):
    rec.update(results[i])

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("\nSaved detailed results to", out_path)


Saved detailed results to sample_comparison_with_metrics.json
